# 15. Apply Model to Train and Test Dashcam

For every image in the "train" and "test" directories, process them through the model in such a way that we divide them into "hits" and "misses" and draw overlays.

The purpose of this notebook is to allow us to inspect the results we get with the latest model on the "train" and especially the "test" dataset, to get an understanding of how it is performing beyond just the mAP and total loss stats.

This is equivalent to notebook 08, but for dashcam footage instead of GSV footage

## Configuration

Any configuration that is required to run this notebook can be customized in the next cell

In [1]:
# Trained detection model name
trained_model_name   = 'centernet_V2_4000'

# Prefix that will be included as a suffix in the label map file and tfrecord train and test files
dataset_version = 'V2'

# Confidence threshold to apply with the model.  The model must be at least
# this confident of a detection for it to count
confidence_threshold = 0.60

## Code

In [2]:
# General imports
import os
import sys

from pathlib import Path

# Make sure local modules can be imported
module_path_root = os.path.abspath(os.pardir)
if module_path_root not in sys.path:
    sys.path.append(module_path_root)

# Get root install path, a level above the minor_thesis folder from GitHub
install_path_root = Path(module_path_root).parent.absolute()

# Import local modules
import osm_gsv_utils.osm_walker as osm_walker
import osm_gsv_utils.gsv_loader as gsv_loader
import tf2_utils.tf2_model_wrapper as tf2_model_wrapper

In [3]:
# Derive paths

# Derived path for main dataset images directory, train images directory, and test images directory
image_train_dir   = os.path.join(install_path_root, 'TensorFlow', 'workspace', 'images', 'train_{0:s}'.format(dataset_version))
image_test_dir    = os.path.join(install_path_root, 'TensorFlow', 'workspace', 'images', 'test_{0:s}'.format(dataset_version))

# Output directory where the detection log and any images with detection overlays will be written
output_directory_train = os.path.join(module_path_root, 'detections', 'train' + '_' + dataset_version)
output_directory_test  = os.path.join(module_path_root, 'detections', 'test'  + '_' + dataset_version)

# Output batch files for each directory
output_batch_train = os.path.join(output_directory_train, 'batch.csv')
output_batch_test  = os.path.join(output_directory_test,  'batch.csv')

# Output CSV file with a log of all detections
detection_log_train_path = os.path.join(output_directory_train, 'detection_log.csv')
detection_log_test_path  = os.path.join(output_directory_train, 'detection_log.csv')

# Create directories if they do not already exist
Path(output_directory_train).mkdir(parents=True, exist_ok=True)
Path(output_directory_test).mkdir(parents=True, exist_ok=True)

# Change directory to make sure the detection model dependencies are found
os.chdir(Path(module_path_root).parent.absolute())
print(os.getcwd())

E:\Release


## Create Batch Files

Create batch files for the "train" and "test" directories that look like the one we will use when processing sample points from a survey area.  These batch files will drive the model to process those directories and output images with overlays, partitioned into "hits" and "misses" directories, etc.

In [4]:
def write_directory_batch_file(input_directory, output_batch_file):
    # List files in the directory
    image_list = [f for f in os.listdir(input_directory) if not f.endswith('.xml')]
    
    file = open(output_batch_file, 'w')
    file.write('lat,lon,bearing,image_path,way_start_id,way_id,node_id,offset_id\n')
    
    for image in image_list:
        file.write('0,0,0,{0:s},0,0,0,0\n'.format(os.path.join(input_directory, image)))

    file.close()
    
write_directory_batch_file(image_train_dir, output_batch_train)
write_directory_batch_file(image_test_dir,  output_batch_test)

## Apply Model to Batch Files

In [5]:
# Initialise model
model_wrapper = tf2_model_wrapper(
    'train', 
    0, 
    image_train_dir, 
    output_directory_train, 
    trained_model_name, 
    version_suffix = dataset_version
)

# Run detections for entire batch
model_wrapper.process_batch_file(
    output_batch_train,
    min_score      = confidence_threshold,
    explicit_files = True,
    progress       = True,
    verbose        = False
)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Output directory for detections: E:\Release\minor_thesis\detections\train_V2
Label Map Path: [TensorFlow\workspace\annotations\label_map_V2.pbtxt]
Latest Checkpoint: ckpt-0


  0%|          | 0/457 [00:00<?, ?it/s]

Instructions for updating:
Use `tf.cast` instead.


'E:\\Release\\minor_thesis\\detections\\train_V2\\detection_log.csv'

In [6]:
# Initialise model
model_wrapper = tf2_model_wrapper(
    'test', 
    0, 
    image_test_dir, 
    output_directory_test, 
    trained_model_name, 
    version_suffix = dataset_version
)

# Run detections for entire batch
model_wrapper.process_batch_file(
    output_batch_test,
    min_score      = confidence_threshold,
    explicit_files = True,
    progress       = True,
    verbose        = False
)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Output directory for detections: E:\Release\minor_thesis\detections\test_V2
Label Map Path: [TensorFlow\workspace\annotations\label_map_V2.pbtxt]
Latest Checkpoint: ckpt-0


  0%|          | 0/136 [00:00<?, ?it/s]

'E:\\Release\\minor_thesis\\detections\\test_V2\\detection_log.csv'